In [1]:
import fiftyone as fo

Migrating database to v0.20.1


In [2]:
import fiftyone as fo
import fiftyone.zoo as foz

In [3]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    max_samples=50,
    shuffle=True,
)

session = fo.launch_app(dataset)

 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 28.7Mb/s]      
Extracting annotations to '/Users/ashwineekumarpandey/fiftyone/coco-2017/raw/instances_val2017.json'
 100% |████████████████████| 50/50 [9.7s elapsed, 0s remaining, 6.0 images/s]      
Writing annotations for 50 downloaded samples to '/Users/ashwineekumarpandey/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/Users/ashwineekumarpandey/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████████| 50/50 [138.9ms elapsed, 0s remaining, 361.9 samples/s] 
Dataset 'coco-2017-validation-50' created


In [24]:
import pandas as pd
import numpy as np
from utils import timer, load_config, get_current_dt, load_npz, f1_score
import itertools
import utils as ut

In [5]:
def load_npz(filepath: str):
    """
    Load numpy array from compressed npz file.

    Args:
        filepath (str): The filepath to load the npz file from.

    Returns:
        Dict[str, List[Tuple[int, int]]]: A dictionary of numpy arrays.
    """
    data = np.load(filepath)
    return data

In [20]:
conf = load_config()

30-Apr-23 02:00:26 - INFO - Starting 'load_config'.
30-Apr-23 02:00:26 - INFO - Finished 'load_config' in 0.0178 secs.


In [21]:
conf.paths.a2_model

'../data/06_models/A2/'

In [22]:
import pickle

my_dict = {1: 'person',
 2: 'bicycle',
 3: 'car',
 4: 'motorcycle',
 5: 'airplane',
 6: 'bs',
 7: 'train',
 8: 'trck',
 9: 'boat',
 10: 'traffic light',
 11: 'fire hydrant',
 12: 'stop sign',
 13: 'parking meter',
 14: 'bench',
 15: 'bird',
 16: 'cat',
 17: 'dog',
 18: 'horse',
 19: 'sheep',
 20: 'cow',
 21: 'elephant',
 22: 'bear',
 23: 'zebra',
 24: 'giraffe',
 25: 'backpack',
 26: 'mbrella',
 27: 'handbag',
 28: 'tie',
 29: 'sitcase',
 30: 'frisbee',
 31: 'skis',
 32: 'snowboard',
 33: 'sports ball',
 34: 'kite',
 35: 'baseball bat',
 36: 'baseball glove',
 37: 'skateboard',
 38: 'srfboard',
 39: 'tennis racket',
 40: 'bottle',
 41: 'wine glass',
 42: 'cp',
 43: 'fork',
 44: 'knife',
 45: 'spoon',
 46: 'bowl',
 47: 'banana',
 48: 'apple',
 49: 'sandwich',
 50: 'orange',
 51: 'broccoli',
 52: 'carrot',
 53: 'hot dog',
 54: 'pizza',
 55: 'dont',
 56: 'cake',
 57: 'chair',
 58: 'coch',
 59: 'potted plant',
 60: 'bed',
 61: 'dining table',
 62: 'toilet',
 63: 'tv',
 64: 'laptop',
 65: 'mose',
 66: 'remote',
 67: 'keyboard',
 68: 'cell phone',
 69: 'microwave',
 70: 'oven',
 71: 'toaster',
 72: 'sink',
 73: 'refrigerator',
 74: 'book',
 75: 'clock',
 76: 'vase',
 77: 'scissors',
 78: 'teddy bear',
 79: 'hair drier',
 80: 'toothbrsh'}

with open('../data/01_raw/coco_labels.pkl', 'wb') as f:
    pickle.dump(my_dict, f)

In [6]:
conf.a1_q1_hyperparams.values()

odict_values([[2, 3, 4], [32], [3], [1, 2], [128], [0.001], [0.9], ['relu', 'selu', 'tanh'], ['same'], [False, True], [True], [0.2], [True], [2]])

In [7]:
param_names = list(conf.a1_q1_hyperparams.keys())
param_names

['num_conv_layers',
 'num_filters',
 'filter_size',
 'num_dense_layers',
 'dense_units',
 'learning_rate',
 'momentum',
 'activation',
 'padding',
 'use_batch_norm',
 'use_dropout',
 'dropout_rate',
 'use_early_stopping',
 'patience']

In [9]:
results_df = pd.DataFrame(columns=["iteration", *param_names, "train_accuracy", "train_f1_score", "val_accuracy", "val_f1_score"])
results_df

,iteration,num_conv_layers,num_filters,filter_size,num_dense_layers,dense_units,learning_rate,momentum,activation,padding,use_batch_norm,use_dropout,dropout_rate,use_early_stopping,patience,train_accuracy,train_f1_score,val_accuracy,val_f1_score


In [11]:
param_values = list(itertools.product(*conf.a1_q1_hyperparams.values()))
param_values

[(2, 32, 3, 1, 128, 0.001, 0.9, 'relu', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 1, 128, 0.001, 0.9, 'relu', 'same', True, True, 0.2, True, 2),
 (2, 32, 3, 1, 128, 0.001, 0.9, 'selu', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 1, 128, 0.001, 0.9, 'selu', 'same', True, True, 0.2, True, 2),
 (2, 32, 3, 1, 128, 0.001, 0.9, 'tanh', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 1, 128, 0.001, 0.9, 'tanh', 'same', True, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'relu', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'relu', 'same', True, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'selu', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'selu', 'same', True, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'tanh', 'same', False, True, 0.2, True, 2),
 (2, 32, 3, 2, 128, 0.001, 0.9, 'tanh', 'same', True, True, 0.2, True, 2),
 (3, 32, 3, 1, 128, 0.001, 0.9, 'relu', 'same', False, True, 0.2, True, 2),
 (3, 32, 3, 1, 128

In [8]:
# generate all possible combinations of hyperparameters
param_values = list(itertools.product(*conf.a1_q1_hyperparams.values()))
param_names = list(conf.a1_q1_hyperparams.keys())
# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=["iteration", *param_names, "train_accuracy", "train_f1_score", "val_accuracy", "val_f1_score"])
results_df

,iteration,num_conv_layers,num_filters,filter_size,num_dense_layers,dense_units,learning_rate,momentum,activation,padding,use_batch_norm,use_dropout,dropout_rate,use_early_stopping,patience,train_accuracy,train_f1_score,val_accuracy,val_f1_score


In [9]:
for i, params in enumerate(param_values):
    param_dict = dict(zip(param_names, params))
    train_accuracy = np.random.rand()
    train_f1_score = np.random.rand()
    val_accuracy = np.random.rand()
    val_f1_score = np.random.rand()
    results_df = results_df.append({
            "iteration": i,
            "num_conv_layers": param_dict["num_conv_layers"],
            'num_filters': param_dict["num_filters"],
            'filter_size': param_dict["filter_size"],
            "num_dense_layers": param_dict["num_dense_layers"],
            "dense_units": param_dict["dense_units"],
            "learning_rate": param_dict["learning_rate"],
            "momentum": param_dict["momentum"],
            "activation": param_dict["activation"],
            "padding": param_dict["padding"],
            "use_batch_norm": param_dict["use_batch_norm"],
            "use_dropout": param_dict["use_dropout"],
            "dropout_rate": param_dict["dropout_rate"],
            "use_early_stopping": param_dict["use_early_stopping"],
            "patience": param_dict["patience"],
            "train_accuracy": train_accuracy, 
            "train_f1_score": train_f1_score,
            "val_accuracy": val_accuracy,
            "val_f1_score": val_f1_score,
        }, ignore_index=True)

/var/folders/0g/blggksdj42z52nv3fy0h5b880000gn/T/ipykernel_51159/3305960176.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0g/blggksdj42z52nv3fy0h5b880000gn/T/ipykernel_51159/3305960176.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0g/blggksdj42z52nv3fy0h5b880000gn/T/ipykernel_51159/3305960176.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0g/blggksdj42z52nv3fy0h5b880000gn/T/ipykernel_51159/3305960176.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [10]:
results_df.sort_values(by="val_f1_score", ascending=False, inplace=True)
secondary = results_df.loc[:, param_names].head(3).to_dict(orient="list")
# results_df

In [15]:
tertiary = results_df.loc[:, param_names].head(1)
tertiary

,num_conv_layers,num_filters,filter_size,num_dense_layers,dense_units,learning_rate,momentum,activation,padding,use_batch_norm,use_dropout,dropout_rate,use_early_stopping,patience
8,2,32,3,2,128,0.001,0.9,selu,same,False,True,0.2,True,2


In [18]:
tertiary["num_conv_layers"].values[0]

2

In [12]:
conf.a1_q1_hyperparams.values()

odict_values([[2, 3, 4], [32], [3], [1, 2], [128], [0.001], [0.9], ['relu', 'selu', 'tanh'], ['same'], [False, True], [True], [0.2], [True], [2]])

In [13]:
secondary.values()

dict_values([[2, 2, 2], [32, 32, 32], [3, 3, 3], [2, 1, 1], [128, 128, 128], [0.001, 0.001, 0.001], [0.9, 0.9, 0.9], ['selu', 'relu', 'relu'], ['same', 'same', 'same'], [False, False, True], [True, True, True], [0.2, 0.2, 0.2], [True, True, True], [2, 2, 2]])

In [ ]:
param_values = list(itertools.product(*secondary))
param_values

In [37]:
param_dict

{'num_conv_layers': 'num_conv_layers',
 'num_filters': 'num_filters',
 'filter_size': 'filter_size',
 'num_dense_layers': 'num_dense_layers',
 'dense_units': 'dense_units',
 'learning_rate': 'learning_rate',
 'momentum': 'momentum',
 'activation': 'activation',
 'padding': 'padding',
 'use_batch_norm': 'use_batch_norm',
 'use_dropout': 'use_dropout',
 'dropout_rate': 'dropout_rate',
 'use_early_stopping': 'use_early_stopping',
 'patience': 'patience'}

In [38]:
data = load_npz(f"{conf.paths.a1_input_data}cifar10.npz")

30-Apr-23 00:57:36 - INFO - Starting 'load_npz'.
30-Apr-23 00:57:36 - INFO - Loaded npz file from ../data/05_model_input/A1/cifar10.npz.
30-Apr-23 00:57:36 - INFO - Finished 'load_npz' in 0.0055 secs.


In [39]:
type(data)

numpy.lib.npyio.NpzFile

In [40]:
data["trainX"].shape

(40000, 32, 32, 3)

In [41]:
data["trainY"].shape

(40000, 10)

In [42]:
data["valY"].shape

(10000, 10)

In [43]:
data["valX"].shape

(10000, 32, 32, 3)